In [1]:
import sys
from pathlib import Path

sys.path.append(Path(".").parent.resolve().as_posix())

import numpy as np
from jesse import helpers, indicators, research

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2024-12-01"),
    helpers.date_to_timestamp("2024-12-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [2]:
from research_tools import TrailingStopLabel

trailing_stop_label = TrailingStopLabel(trading_1m, n_bar=15, min_r=0)
trailing_stop_label.labels.shape

(43200,)

In [3]:
trading_1m.shape

(43200, 6)

In [4]:
np.unique(trailing_stop_label.labels, return_counts=True)

(array([-1,  0,  1], dtype=int8), array([15416, 12970, 14814]))

In [5]:
np.unique(trailing_stop_label.bar_duration, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]),
 array([12970,  7495,  7042,  4895,  3344,  2324,  1515,  1060,   751,
          549,   387,   262,   223,   152,   122,    78,    31]))

In [6]:
real_r = np.abs(trailing_stop_label.realized_r)
real_r = real_r[real_r > 0]
real_r.min()

0.00015462648762243347

In [7]:
real_r.max()

0.035923227998711504

In [8]:
from custom_indicators.td_sequential import td_sequential

# timestamp, open, close, high, low, volume
buy_count, sell_count = td_sequential(
    trading_1m, sequential=True, aggressive=True, stealth_actions=True
)

In [9]:
np.unique(buy_count, return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14.]),
 array([11722,  1178,  1106,  1111,  1093,  1125,  1057,   940,   866,
          838,   736,   729,   704,  8455, 11540]))